## **ETAPA 1 - Coleta dos dados da API do Banco Central**

In [ ]:
anual = "https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativasMercadoAnuais?$top=1000&$format=json&$select=Indicador,IndicadorDetalhe,Data,DataReferencia,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo"

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
def requisicao_api(link):
    resposta = requests.get(link)

    if resposta.status_code == 200:
        dados = resposta.json()
        print('Status Code:', resposta.status_code)

        with open('resultado_anual.json', 'w', encoding='utf-8') as arquivo:
            json.dump(dados, arquivo, ensure_ascii=False, indent=4)
    else:
        print('Erro na requisição. Status Code:', resposta.status_code)

In [ ]:
requisicao_api(anual)

with open("resultado_anual.json", "r", encoding="utf-8") as f:
    dados = json.load(f)

registros = dados["value"]

df = pd.DataFrame(registros)
df.head()

Status Code: 200


,Indicador,IndicadorDetalhe,Data,DataReferencia,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo
0,Balança comercial,Exportações,2017-06-19,2020,230.92,223.81,17.37,198.0,261.03,16.0,0
1,Balança comercial,Exportações,2017-06-19,2019,218.80,217.00,12.87,195.0,239.60,18.0,0
2,Balança comercial,Exportações,2017-06-19,2021,242.69,233.65,23.61,200.0,281.50,14.0,0
3,Balança comercial,Exportações,2017-06-19,2018,210.98,210.00,11.89,191.0,244.42,29.0,0
4,Balança comercial,Exportações,2017-06-19,2017,207.24,209.00,7.72,192.6,225.00,35.0,0


## **ETAPA 2 – Processamento de dados com Spark**

In [ ]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-11-jdk -qq > /dev/null
!pip install -q pyspark

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] += os.pathsep + os.path.join(os.environ["JAVA_HOME"], "bin")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round

spark = SparkSession.builder \
    .appName("BCB_API_PySpark") \
    .master("local[*]") \
    .getOrCreate()

In [ ]:
df_spark = spark.createDataFrame(registros)

df_spark.printSchema()

root
 |-- Data: string (nullable = true)
 |-- DataReferencia: string (nullable = true)
 |-- DesvioPadrao: double (nullable = true)
 |-- Indicador: string (nullable = true)
 |-- IndicadorDetalhe: string (nullable = true)
 |-- Maximo: double (nullable = true)
 |-- Media: double (nullable = true)
 |-- Mediana: double (nullable = true)
 |-- Minimo: double (nullable = true)
 |-- baseCalculo: long (nullable = true)
 |-- numeroRespondentes: long (nullable = true)



In [ ]:
df_spark.show(5, truncate=False)

+----------+--------------+------------+-----------------+----------------+------+------+-------+------+-----------+------------------+
|Data      |DataReferencia|DesvioPadrao|Indicador        |IndicadorDetalhe|Maximo|Media |Mediana|Minimo|baseCalculo|numeroRespondentes|
+----------+--------------+------------+-----------------+----------------+------+------+-------+------+-----------+------------------+
|2017-06-19|2020          |17.37       |Balança comercial|Exportações     |261.03|230.92|223.81 |198.0 |0          |16                |
|2017-06-19|2019          |12.87       |Balança comercial|Exportações     |239.6 |218.8 |217.0  |195.0 |0          |18                |
|2017-06-19|2021          |23.61       |Balança comercial|Exportações     |281.5 |242.69|233.65 |200.0 |0          |14                |
|2017-06-19|2018          |11.89       |Balança comercial|Exportações     |244.42|210.98|210.0  |191.0 |0          |29                |
|2017-06-19|2017          |7.72        |Balança 

## **ETAPA 3 – Transformações com PySpark**

In [ ]:
from pyspark.sql import functions as F

# 1. Selecionar colunas específicas
df_select = df_spark.select("Indicador", "DataReferencia", "Media", "Mediana")
df_select.show(5, truncate=False)

+-----------------+--------------+------+-------+
|Indicador        |DataReferencia|Media |Mediana|
+-----------------+--------------+------+-------+
|Balança comercial|2020          |230.92|223.81 |
|Balança comercial|2019          |218.8 |217.0  |
|Balança comercial|2021          |242.69|233.65 |
|Balança comercial|2018          |210.98|210.0  |
|Balança comercial|2017          |207.24|209.0  |
+-----------------+--------------+------+-------+
only showing top 5 rows



In [ ]:
# 2. Criar nova coluna: diferença entre Média e Mediana
df_novo = df_spark.withColumn("Diferenca", F.col("Media") - F.col("Mediana"))
df_novo.select("Indicador", "DataReferencia", "Media", "Mediana", "Diferenca").show(5, truncate=False)

+-----------------+--------------+------+-------+------------------+
|Indicador        |DataReferencia|Media |Mediana|Diferenca         |
+-----------------+--------------+------+-------+------------------+
|Balança comercial|2020          |230.92|223.81 |7.109999999999985 |
|Balança comercial|2019          |218.8 |217.0  |1.8000000000000114|
|Balança comercial|2021          |242.69|233.65 |9.039999999999992 |
|Balança comercial|2018          |210.98|210.0  |0.9799999999999898|
|Balança comercial|2017          |207.24|209.0  |-1.759999999999991|
+-----------------+--------------+------+-------+------------------+
only showing top 5 rows



In [ ]:
# 3. Filtrar registros: apenas projeções com mais de 50 respondentes
df_filtrado = df_spark.filter(F.col("numeroRespondentes") > 50)
df_filtrado.select("Indicador", "numeroRespondentes").show(5, truncate=False)

+---------+------------------+
|Indicador|numeroRespondentes|
+---------+------------------+
+---------+------------------+



In [ ]:
# 4. Ordenar por maior média
df_ordenado = df_spark.orderBy(F.col("Media").desc())
df_ordenado.select("Indicador", "Media", "Mediana").show(10, truncate=False)

+-----------------+------+-------+
|Indicador        |Media |Mediana|
+-----------------+------+-------+
|Balança comercial|315.26|315.43 |
|Balança comercial|306.95|314.37 |
|Balança comercial|292.9 |295.5  |
|Balança comercial|273.67|275.0  |
|Balança comercial|260.43|261.38 |
|Balança comercial|257.99|251.65 |
|Balança comercial|256.48|255.0  |
|Balança comercial|255.16|253.51 |
|Balança comercial|255.16|253.51 |
|Balança comercial|255.16|253.51 |
+-----------------+------+-------+
only showing top 10 rows



In [ ]:
# 5. Selecionar e renomear colunas
df_renomeado = df_spark.select(
    F.col("Indicador").alias("Nome_Indicador"),
    F.col("Media").alias("Valor_Medio"),
    F.col("Mediana").alias("Valor_Mediano")
)
df_renomeado.show(5, truncate=False)

+-----------------+-----------+-------------+
|Nome_Indicador   |Valor_Medio|Valor_Mediano|
+-----------------+-----------+-------------+
|Balança comercial|230.92     |223.81       |
|Balança comercial|218.8      |217.0        |
|Balança comercial|242.69     |233.65       |
|Balança comercial|210.98     |210.0        |
|Balança comercial|207.24     |209.0        |
+-----------------+-----------+-------------+
only showing top 5 rows



In [ ]:
# 6. Criar uma coluna categórica com base no valor da média
df_categoria = df_spark.withColumn(
    "Categoria_Media",
    F.when(F.col("Media") > 5, "Alta Projeção")
     .otherwise("Baixa Projeção")
)
df_categoria.select("Indicador", "Media", "Categoria_Media").show(5, truncate=False)

+-----------------+------+---------------+
|Indicador        |Media |Categoria_Media|
+-----------------+------+---------------+
|Balança comercial|230.92|Alta Projeção  |
|Balança comercial|218.8 |Alta Projeção  |
|Balança comercial|242.69|Alta Projeção  |
|Balança comercial|210.98|Alta Projeção  |
|Balança comercial|207.24|Alta Projeção  |
+-----------------+------+---------------+
only showing top 5 rows



## **ETAPA 4 – Ações com o DataFrame Spark**

In [ ]:
# 1. Mostrar registros
df_spark.show(5, truncate=False)

+----------+--------------+------------+-----------------+----------------+------+------+-------+------+-----------+------------------+
|Data      |DataReferencia|DesvioPadrao|Indicador        |IndicadorDetalhe|Maximo|Media |Mediana|Minimo|baseCalculo|numeroRespondentes|
+----------+--------------+------------+-----------------+----------------+------+------+-------+------+-----------+------------------+
|2017-06-19|2020          |17.37       |Balança comercial|Exportações     |261.03|230.92|223.81 |198.0 |0          |16                |
|2017-06-19|2019          |12.87       |Balança comercial|Exportações     |239.6 |218.8 |217.0  |195.0 |0          |18                |
|2017-06-19|2021          |23.61       |Balança comercial|Exportações     |281.5 |242.69|233.65 |200.0 |0          |14                |
|2017-06-19|2018          |11.89       |Balança comercial|Exportações     |244.42|210.98|210.0  |191.0 |0          |29                |
|2017-06-19|2017          |7.72        |Balança 

In [ ]:
# 2. Contar número total de registros
print("Total de registros:", df_spark.count())

Total de registros: 1000


In [ ]:
# 3. Descrever estatísticas básicas das colunas numéricas
# Primeiro, converter colunas para tipo numérico
df_numerico = df_spark.withColumns({
    "Media": F.col("Media").cast("float"),
    "Mediana": F.col("Mediana").cast("float"),
    "DesvioPadrao": F.col("DesvioPadrao").cast("float"),
    "Minimo": F.col("Minimo").cast("float"),
    "Maximo": F.col("Maximo").cast("float"),
})

df_numerico.describe(["Media", "Mediana", "DesvioPadrao", "Minimo", "Maximo"]).show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+
|summary|             Media|          Mediana|     DesvioPadrao|           Minimo|           Maximo|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1000|             1000|             1000|             1000|             1000|
|   mean|  83.1006402053833|82.93738995742798|4.875440007179976|74.22622000885009| 93.2695802230835|
| stddev|47.543707665105416|47.71294778532735|4.667182066555187|39.47506042061932|54.72666875416481|
|    min|             55.83|            55.13|              0.0|              0.0|             59.5|
|    max|            315.26|           315.43|            48.49|            240.3|            420.0|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+



In [ ]:
# 4. Calcular médias das colunas de interesse
df_numerico.select(
    F.mean("Media").alias("Média_Geral"),
    F.mean("Mediana").alias("Mediana_Geral")
).show()

+----------------+-----------------+
|     Média_Geral|    Mediana_Geral|
+----------------+-----------------+
|83.1006402053833|82.93738995742798|
+----------------+-----------------+



In [ ]:
# 5. Obter o registro com maior valor médio
df_numerico.orderBy(F.col("Media").desc()).limit(5).show(truncate=False)

+----------+--------------+------------+-----------------+----------------+------+------+-------+------+-----------+------------------+
|Data      |DataReferencia|DesvioPadrao|Indicador        |IndicadorDetalhe|Maximo|Media |Mediana|Minimo|baseCalculo|numeroRespondentes|
+----------+--------------+------------+-----------------+----------------+------+------+-------+------+-----------+------------------+
|2011-08-08|2014          |37.4        |Balança comercial|Exportações     |396.0 |315.26|315.43 |240.3 |0          |NULL              |
|2011-05-30|2015          |48.49       |Balança comercial|Exportações     |420.0 |306.95|314.37 |205.8 |0          |NULL              |
|2011-05-30|2014          |36.89       |Balança comercial|Exportações     |370.0 |292.9 |295.5  |225.9 |0          |NULL              |
|2011-05-30|2013          |25.18       |Balança comercial|Exportações     |325.0 |273.67|275.0  |235.0 |0          |NULL              |
|2011-05-30|2012          |21.87       |Balança 

In [ ]:
# 6. Calcular soma de médias agrupadas pela categoria
df_categoria.groupBy("Categoria_Media").agg(
    F.sum("Media").alias("Soma_Media"),
    F.avg("Mediana").alias("Media_Mediana")
).show()

+---------------+-----------------+-----------------+
|Categoria_Media|       Soma_Media|    Media_Mediana|
+---------------+-----------------+-----------------+
|  Alta Projeção|83100.63999999998|82.93738999999998|
+---------------+-----------------+-----------------+

